<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [72]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [73]:
import numpy as np
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
#print(Hardwood_Sawlog_Data)

**SAWMILL CLASS**

In [84]:
#SawMill Object
class Mill:
  m = GEKKO(remote=True)

  def __init__(self, Id, Wood, Prod, Dis, Dem=0):
    self.Id = Id
    self.Wood_type =Wood
    self.Prod_type = Prod
    self.Dist = Dis
    self.Demand = Dem
    #print("Mill: ",self.Id, " Created")

  def Harvest_Close(self, Available, radio, Dem = 0):
    if (Dem>0):
      self.Demand = Dem
    
    Num = 100
  
    H = m.Array(m.Var,(Num),value = 0)

    for i in range(Num):
      H[i].lower = 0
      H[i].upper = int(Available[i])
      
    prod0 = m.Intermediate(m.sum([self.Dist[i]*H[i] for i in range(0,len(H))]))
    m.Equation(self.Demand==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=True,debug=True)
    print(H)
    return(H)
    print("Error. Not enough cells")

In [96]:
class Inventory:
  def __init__(self, Av, Dist):
    self.Available = []
    self.Available.append(Av)
    print(self.Available)
    self.Dist = Dist
    self.Mills = []
  
  def Add_Mill(self, m):
    self.Mills.append(m)

  
  def Harvest_Mill(self,t, j, Radio =10):
   # Av = self.Cell_Less(Radio)
    ans = self.Mills[j].Harvest_Close(self.Available[t].SumOfTotal,Radio,self.Mills[j].Demand)

  
  def Harvest_All_Mills(self, Radio =10):
    ans = self.Mills[j].Harvest_Close(self.Available,Radio,self.Mills[j].Demand)

  
  def Cell_Less(self, Id_Mill, Time, Radio=10):
    #Given a Time and Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    Disp =[]
    for j in range(len(Ans)):
      if not(self.Available[0][self.Available[0]["Unique_ID"]==Ans['DEST'][j]].empty):
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans['DEST'][j]].SumOfTotal))
      else:
        Disp.append(0)
    Ans["AVAIL"] = Disp
    #print(Ans)  
    return(Ans)
    
  
  def runInParallel(*fns):
    proc = []
    for fn in fns:
      p = Process(target=fn)
      p.start()
      proc.append(p)
    for p in proc:
      p.join()



**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [99]:
# Main Program
Num_Cell= 10000
D = np.zeros(Num_Cell)
Demand = 20000000 

for i in range(Num_Cell):
  D[i] = i

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
#print(Hardwood_Sawlog.Available[0]["Unique_ID"]==74031)

Hardwood_Sawlog.Cell_Less(54549,0,10)
ma = Hardwood_Sawlog_Data['Unique_ID'].unique()
#print(Hardwood_Sawlog_Data)

#a = Mill(74031,0,0,Hardwood_Sawlog.Dist.DIST,1000000)
#a = Mill(Mills_Data['Unique_ID'][0],Mills_Data['plant'][0],Mills_Data['s_h'][0],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][0])
#a.Harvest_Close(Hardwood_Sawlog_Data.SumOfTotal, 10, 1000000)

for k in range(10):
  
  #Hardwood_Sawlog.Mills.append(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][k]))
  Hardwood_Sawlog.Add_Mill(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][k]))
  print("Cosechando Aserradero ", k)
  Hardwood_Sawlog.Harvest_Mill(0,k)

[      Unique_ID   SumOfTotal
0         74031  1669305.768
1         75559  2815597.681
2         86608  1072054.583
3         68554  3633457.723
4         68082  2210296.267
...         ...          ...
7758      76917  4321040.095
7759      61910  1815435.841
7760      76697  4277713.205
7761      75323  3729925.067
7762      54749  2268199.612

[7763 rows x 2 columns]]
Cosechando Aserradero  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


apm 34.91.131.11_gk_model13 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           14
   Constants    :            0
   Variables    :         2414
   Intermediates:            7
   Connections  :         1414
   Equations    :          721
   Residuals    :          714
 
 Number of state variables:           2414
 Number of total equations: -          721
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :           1693
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con